In [1]:
import pandas as pd
import random
import numpy as np
import math
import sep
import cv2

from numpy.lib import recfunctions as rfn
from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord


In [2]:
def readPSFfromFile(file):

    #psf1='/Users/chyan/PythonCode/Instrument/StarTracker/psf/im0_0deg.TXT'
    with open(file,'r') as f:
        lines = f.readlines()[19:]

    image=np.array([])
    for l in lines:
        if image.size == 0:
            image = np.array(l.replace("\n","").split()).astype(float)
        else:
            image = np.vstack((image, np.array(l.replace("\n","").split()).astype(float)))
    
    image=image/np.sum(image)
    return image

In [74]:
def findSpot(data, sigma):
    image=data
    #m, s = np.mean(image), np.std(image)
    bkg = sep.Background(image, bw=32, bh=32, fw=3, fh=3)
    objs = sep.extract(image-bkg, sigma, err=bkg.globalrms)
    aper_radius=3
    
    # Calculate the Kron Radius
    kronrad, krflag = sep.kron_radius(image, objs['x'], objs['y'], \
        objs['a'], objs['b'], objs['theta'], aper_radius)

    r_min = 3
    use_circle = kronrad * np.sqrt(objs['a'] * objs['b'])
    cinx=np.where(use_circle <= r_min)
    einx=np.where(use_circle > r_min)

    # Calculate the equivalent of FLUX_AUTO
    flux, fluxerr, flag = sep.sum_ellipse(image, objs['x'][einx], objs['y'][einx], \
        objs['a'][einx], objs['b'][einx], objs['theta'][einx], 2.5*kronrad[einx],subpix=1)		

    cflux, cfluxerr, cflag = sep.sum_circle(image, objs['x'][cinx], objs['y'][cinx],
                                    objs['a'][cinx], subpix=1)

    # Adding half pixel to measured coordinate.  
    objs['x'] =  objs['x']+0.5
    objs['y'] =  objs['y']+0.5

    objs['flux'][einx]=flux
    objs['flux'][cinx]=cflux


    r, flag = sep.flux_radius(image, objs['x'], objs['y'], \
        6*objs['a'], 0.3,normflux=objs['flux'], subpix=5)

    flag |= krflag
 
    objs=rfn.append_fields(objs, 'r', data=r, usemask=False)

    objects=objs[:]
    
    return objects

def polar(x, y):
    theta = np.arctan2(y, x)* 180 / np.pi
    rho = np.hypot(x, y)
    ind=np.where(theta<0)
    
    theta[ind]=theta[ind]+360
    return theta, rho


def getRGBvalue(photon):
    
    if photon < 1:
        return 0,0,0
    else:
        count = [e for e in range(256)]
        r=[]
        g=[]
        b=[]


        for i in count:
            if i is 0:
                r.append(0)
                g.append(0)
                b.append(0)
            elif i < 50:
                r.append(10**(-3.358+2.878*np.log10(i)))
                g.append(10**(-3.582+3.016*np.log10(i)))
                b.append(10**(-5.943+4.539*np.log10(i)))
            else:
                r.append(10**(-2.175+2.138*np.log10(i)))
                g.append(10**(-2.206+2.167*np.log10(i)))
                b.append(10**(-2.433+2.207*np.log10(i)))

        # scale the count, so that the maximum of R+G+B = 2048
        d = {'count': count, 
             'R':2048*(r/(r[255]+g[255]+g[255])), 
             'G':2048*(g/(r[255]+g[255]+g[255])), 
             'B':2048*(b/(r[255]+g[255]+g[255]))}
        df = pd.DataFrame(data=d)

        idx=np.random.uniform(0, 255.0,3)
        while abs(photon-df["R"][idx[0].astype(int)]-df["G"][idx[1].astype(int)]-df["B"][idx[2].astype(int)]) > 1:
            idx=np.random.uniform(0, 255,3)

        #print(photon-df["R"][idx[0].astype(int)]-df["G"][idx[1].astype(int)]-df["B"][idx[2].astype(int)])
        #print(f'{rid} {df["R"][rid]} {gid} {df["G"][gid]} {bid} {df["B"][bid]} ')

        return idx[0].astype(int),idx[1].astype(int),idx[2].astype(int)



In [105]:
rgb_value=[]
for i in range(1280):
    rgb_value.append(getRGBvalue(i))
    #print(i,getRGBvalue(i))

In [4]:
df = pd.read_csv('Catalog/gsc.all', sep='\s+')

In [10]:
ra = random.uniform(-90, 90)
dec = random.uniform(0, 360)


psf0=readPSFfromFile('/Users/chyan/PythonCode/Instrument/StarTracker/psf/im0_0deg.TXT')
psf5=readPSFfromFile('/Users/chyan/PythonCode/Instrument/StarTracker/psf/im0_5deg.TXT')
psf8=readPSFfromFile('/Users/chyan/PythonCode/Instrument/StarTracker/psf/im0_8deg.TXT')

# Give a exposure time
exptime = 0.1 #

In [11]:
a=map(list,zip(*[df['dec'].values,df['ra'].values]))
coord=np.array([])
for _ in a:
    if coord.size == 0:
        coord = np.array([_])
    else:
        coord = np.vstack((coord,_))

In [12]:
# Create a new WCS object.  The number of axes must be set
# from the start
w = wcs.WCS(naxis=2)

# Set up an orthographic projection
# Vector properties may be set with Python lists, or Numpy arrays
w.wcs.crpix = [640,512]
w.wcs.cdelt = np.array([-0.015972223, 0.015972223])
w.wcs.crval = [dec, ra]
w.wcs.ctype = ["RA---SIN", "DEC--SIN"]
#w.wcs.set_pv([(2, 1, 45.0)])

pixcrd = np.array([[0, 0], [24, 38], [45, 98],[640,512]], np.float_)

# Convert pixel coordinates to world coordinates
world = w.wcs_pix2world(pixcrd, 1)
#print(world)

# Convert the same coordinates back to pixel coordinates.
pixcrd2 = w.wcs_world2pix(coord, 1)
#print(pixcrd2)


# Putting X, Y to data frame
df['x']=pixcrd2[:,0]
df['y']=pixcrd2[:,1]

# Converting Vmag to ABmag. Note the lambda = 0.5456 micron 
df['ABmag']=0.02+df['Vmag']
df['Jy']=10**((df['ABmag']-8.9)/(-2.5))
df['theta'],df['rho']=polar(df['x'].values-640,df['y'].values-512)


# This is the photon energy at 0.5456 micron 
e_photon = 3.61e-19 # J 

# Assuming the bandwidth of filter V
frequency = 400.8e12 # Hz

# Collecting area
psize = math.pi*0.005**2

# QE
qe=0.4

# Gain e-/DN
gain=10.2 

# Transmission
tran= 0.94

df['Nphoton']=(df['Jy']*10e-26/e_photon)*frequency*psize*qe*tran*exptime/gain


# Now, write out the WCS object as a FITS header
header = w.to_header()

# header is an astropy.io.fits.Header object.  We can use it to create a new
# PrimaryHDU and write it to a file.
#hdu = fits.PrimaryHDU(header=header)

# Selecting stars in the field
stars=df[(50 < df['x']) & (df['x'] < 1230) & (df['y'] > 50) & (df['y'] < 974)]



/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in less


In [13]:
stars

,dec,ra,Vmag,source,x,y,ABmag,Jy,theta,rho,Nphoton
10499,182.641266,-37.870554,6.09,T,1214.748722,485.335605,6.11,13.061709,357.343774,575.366912,419.853778
10505,183.119655,-33.404987,7.24,T,1223.300057,765.580615,7.26,4.528976,23.496244,636.036230,145.578777
10506,183.268753,-31.518368,7.69,T,1227.841186,883.338385,7.71,2.992265,32.280540,695.305297,96.182944
10507,183.302638,-33.333104,7.64,T,1214.390945,771.176281,7.66,3.133286,24.285815,630.156570,100.715906
10508,183.303673,-34.125218,6.62,T,1209.060777,722.043529,6.64,8.016781,20.259387,606.587547,257.690297
10509,183.403382,-33.793054,6.47,T,1206.194238,743.264590,6.49,9.204496,22.217781,611.603815,295.868047
10511,183.503141,-32.387042,7.47,T,1210.142615,831.040177,7.49,3.664376,29.230486,653.337000,117.787191
10512,183.525448,-36.729022,7.22,T,1180.027171,561.583703,7.24,4.613176,5.246021,542.298708,148.285288
10513,183.528857,-36.556429,7.83,T,1181.070119,572.335487,7.85,2.630268,6.362846,544.423773,84.546973
10514,183.574972,-33.779107,7.40,T,1197.516993,745.166572,7.42,3.908409,22.695816,604.311052,125.631361


In [37]:
skyimage = np.zeros((1024, 1280))

# Loop through star table 
for index, row in stars.iterrows():
    center = SkyCoord(dec, ra, frame='icrs',unit='deg')
    target = SkyCoord(row['dec'], row['ra'], frame='icrs',unit='deg')
    sep = center.separation(target)
    #print(sep.degree)
    if sep.degree < 3:
        psf = psf0
    if 3 <= sep.degree < 8:
        psf = psf5
    if sep.degree >= 8:
        psf = psf8
    #try:
    #print(y-50,y+50,x-50,x+50)
    
    x=np.floor(row['x']).astype(int)
    y=np.floor(row['y']).astype(int)
    point = row['Nphoton']*psf
    #print(point.shape)
    #skyimage[y,x]=skyimage[y,x]+row['Nphoton']
    skyimage[y-50:y+50,x-50:x+50]=skyimage[y-50:y+50,x-50:x+50]+point
    #except:
    
skyimage[skyimage>1024] = 1024

In [21]:
np.max(skyimage.astype(int))

1024

In [109]:
rgb_value[-1]

(232, 172, 240)

In [108]:
img = np.zeros((skyimage.shape[0], skyimage.shape[1], 3), dtype = "uint8")



for i in range(skyimage.shape[0]):
    for j in range(skyimage.shape[1]):
        #print(skyimage[i, j].astype('int'))
        img[i,j,0]=rgb_value[skyimage[i, j].astype('int')][2]
        img[i,j,1]=rgb_value[skyimage[i, j].astype('int')][1]
        img[i,j,2]=rgb_value[skyimage[i, j].astype('int')][0]
        #if skyimage[i, j] > 1:
        
        #print(i,j,r,g,b,skyimage[i, j])
    #print(i)
cv2.imwrite('test.tiff',img)        

True

In [86]:
getRGBvalue(3)

(6, 10, 28)

In [81]:
img = np.zeros((1024, 1280, 3), dtype = "uint8")
img[:,:,0]=255
cv2.imwrite('test.tiff',img)
 

True

In [228]:
count = [e for e in range(256)]
r=[]
g=[]
b=[]


for i in count:
    if i is 0:
        r.append(0)
        g.append(0)
        b.append(0)
    elif i < 50:
        r.append(10**(-3.358+2.878*np.log10(i)))
        g.append(10**(-3.582+3.016*np.log10(i)))
        b.append(10**(-5.943+4.539*np.log10(i)))
    else:
        r.append(10**(-2.175+2.138*np.log10(i)))
        g.append(10**(-2.206+2.167*np.log10(i)))
        b.append(10**(-2.433+2.207*np.log10(i)))

# scale the count, so that the maximum of R+G+B = 1024
d = {'count': count, 
     'R':2048*(r/(r[255]+g[255]+g[255])), 
     'G':2048*(g/(r[255]+g[255]+g[255])), 
     'B':2048*(b/(r[255]+g[255]+g[255]))}
df = pd.DataFrame(data=d)

,count,R,G,B
0,0,0.000000,0.000000,0.000000e+00
1,1,0.000151,0.000090,3.924235e-07
2,2,0.001109,0.000729,9.122841e-06
3,3,0.003564,0.002476,5.746565e-05
4,4,0.008156,0.005896,2.120827e-04
5,5,0.015502,0.011557,5.839556e-04
6,6,0.026198,0.020029,1.335929e-03
7,7,0.040827,0.031884,2.689397e-03
8,8,0.059958,0.047695,4.930380e-03
9,9,0.084152,0.068038,8.415147e-03


In [231]:
photon = 900






rvalue,gvalue,bvalue=getRGBvalue(photon)
#print(g_ret-df['B'][bid])
#bid=int(np.random.uniform(0, 255.0))

#photon-df['G'][gid]-df['B'][bid]

In [232]:
print(f'{rid} {df["R"][rid]} {gid} {df["G"][gid]} {bid} {df["B"][bid]} ')
print(photon-df["R"][rid]-df["G"][gid]-df["B"][bid])

104 94.45006184341078 245 644.3292870515874 150 161.2197992134541 
0.0008518915477111477


In [176]:
idx=np.random.uniform(0, 255.0,3)

In [204]:
photon = 760
idx=np.random.uniform(0, 255.0,3)
while abs(photon-df["R"][idx[0].astype(int)]-df["G"][idx[1].astype(int)]-df["B"][idx[2].astype(int)]) > 0.1:
    idx=np.random.uniform(0, 255.0,3)
print(photon-df["R"][idx[0].astype(int)]-df["G"][idx[1].astype(int)]-df["B"][idx[2].astype(int)])
print(f'{rid} {df["R"][rid]} {gid} {df["G"][gid]} {bid} {df["B"][bid]} ')

-0.016566023040923028
224 243.5483741656516 65 18.169300820407635 201 153.7831208871312 


In [ ]:
for _ in df[[1,2]]

In [ ]:
df

In [ ]:
w.wcs_world2pix([[250.427083,1.181111]],1)

In [12]:
from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text

hover = HoverTool(tooltips=[
                ("index", "$index"),
                ("data (x,y)", "($x, $y)"),
                ("canvas (x,y)", "($sx, $sy)"),
                ("value", "@image"),
                ])

TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset',hover]

cmap=LinearColorMapper(palette="Viridis256", low=0.001, high=0.01)


p = figure(x_range=(0, 100), y_range=(0, 100),plot_width=500, plot_height=500,tools=TOOLS)
p.image(image=[psf8], x=0, y=0, dw=100, dh=100, color_mapper=cmap)


show(p)

In [ ]:
np.random.random((10,10))

In [ ]:
noise=np.random.normal(0, 1.0, 1024*1280).reshape(1024,1280)

In [34]:
def rotateImage(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result


In [37]:
hdul = fits.open('psf.fits')
image=hdul[0].data
objs=findSpot(image.astype(np.float32),30)
psf = pd.DataFrame(data={'x':np.floor(objs['x']).astype(int),
                         'y':np.floor(objs['y']).astype(int),
                         'r':np.floor(6*objs['r']).astype(int)})
psf['x0']=psf['x']-psf['r']
psf['x1']=psf['x']+psf['r']
psf['y0']=psf['y']-psf['r']
psf['y1']=psf['y']+psf['r']

subimage=image[psf['y0'][2]:psf['y1'][2],psf['x0'][2]:psf['x1'][2]]

aa=rotateImage(subimage,35)

In [144]:
def getPSFfromImage(theta, rho):
    hdul = fits.open('psf.fits')
    image=hdul[0].data
    objs=findSpot(image.astype(np.float32),30)
    psf = pd.DataFrame(data={'x':np.floor(objs['x']).astype(int),
                             'y':np.floor(objs['y']).astype(int),
                             'r':np.floor(8*objs['r']).astype(int)})
    psf['x0']=psf['x']-psf['r']
    psf['x1']=psf['x']+psf['r']
    psf['y0']=psf['y']-psf['r']
    psf['y1']=psf['y']+psf['r']
    
    
    psf['theta'],psf['rho']=polar(psf['x'].values-640,psf['y'].values-512)
    
    # selecting closet PSF
    ind=np.where((psf['rho']-rho).abs() == min((psf['rho']-rho).abs()))
    
    # Getting sub-image
    x0=psf['x0'][ind[0]].values[0]
    x1=psf['x1'][ind[0]].values[0]
    y0=psf['y0'][ind[0]].values[0]
    y1=psf['y1'][ind[0]].values[0]
    
    subimage=image[y0:y1,x0:x1]

    
    
    # calculating the angle difference
    delta_angle = psf['theta'][ind[0]] - theta
    
    print(psf['theta'][ind[0]],psf['rho'][ind[0]], delta_angle.values[0])


    #Rotate the subimage
    rot_subimage=np.array(
        PIL.Image.fromarray(subimage).rotate(-delta_angle.values[0], resample=PIL.Image.BILINEAR)
        )
    
    # Cast the array to float and normalize to unity
    
    psfimg=rot_subimage.astype(np.float32)
    psfimg[:]=psfimg[:]/np.sum(psfimg)
    
    #print(psfimg)
    return psfimg

In [145]:
getPSFfromImage(278.304764,75.457158)

21    141.679886
Name: theta, dtype: float64 21    79.025312
Name: rho, dtype: float64 -136.62487793677744


array([[0.        , 0.        , 0.00622038, 0.00622038, 0.00622038,
        0.00622038, 0.00503555, 0.00473934, 0.        , 0.        ],
       [0.        , 0.00799763, 0.00859005, 0.00947867, 0.0103673 ,
        0.00888626, 0.00651659, 0.00592417, 0.00533175, 0.        ],
       [0.00770142, 0.00947867, 0.01303317, 0.0207346 , 0.02162322,
        0.0174763 , 0.01125592, 0.007109  , 0.00592417, 0.00533175],
       [0.00770142, 0.01214455, 0.0242891 , 0.03317536, 0.03554503,
        0.03021327, 0.01866114, 0.0103673 , 0.00651659, 0.00503555],
       [0.00770142, 0.01362559, 0.02488152, 0.03969194, 0.03732228,
        0.03228673, 0.02310427, 0.01184834, 0.007109  , 0.00473934],
       [0.0068128 , 0.0103673 , 0.01925355, 0.02695498, 0.03169431,
        0.02813981, 0.0174763 , 0.0103673 , 0.00651659, 0.00562796],
       [0.00533175, 0.00799763, 0.01095972, 0.01481043, 0.01629147,
        0.01481043, 0.01125592, 0.00799763, 0.00562796, 0.00444313],
       [0.00444313, 0.00562796, 0.0068128

In [89]:
ind=np.where((psf['rho']-89.35).abs() == min((psf['rho']-89.35).abs()))
#psf['rho'][min((psf['rho']-89.35).abs()) == (psf['rho']-89.35).abs()
psf['rho'][ind[0]]

15    82.462113
Name: rho, dtype: float64

In [82]:
psf

,x,y,r,x0,x1,y0,y1,theta,rho
0,974,165,5,969,979,160,170,313.906381,481.627449
1,958,181,5,953,963,176,186,313.852472,459.004357
2,939,200,4,935,943,196,204,313.781125,432.140024
3,920,219,3,917,923,216,222,313.700319,405.276449
4,904,236,4,900,908,232,240,313.726970,381.931931
5,885,255,4,881,889,251,259,313.630641,355.069007
6,866,274,3,863,869,271,277,313.518543,328.207252
7,846,293,4,842,850,289,297,313.247970,300.660939
8,827,312,4,823,831,308,316,313.076059,273.804675
9,807,332,4,803,811,328,336,312.854476,245.538184


In [124]:
ii=PIL.Image.fromarray(image).rotate(45, resample=PIL.Image.BILINEAR)

In [125]:
ii.show()

In [13]:
from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, show, output_file
import bokeh.palettes
from bokeh.layouts import column
from bokeh.models import HoverTool, ColumnDataSource, LinearColorMapper
from bokeh.models.glyphs import Text

hover = HoverTool(tooltips=[
                ("index", "$index"),
                ("data (x,y)", "($x, $y)"),
                ("canvas (x,y)", "($sx, $sy)"),
                ("value", "@image"),
                ])

TOOLS = ['pan','box_zoom','wheel_zoom', 'save' ,'reset',hover]

cmap=LinearColorMapper(palette="Viridis256", low=0.001, high=500)


p1 = figure(x_range=(0, 1280), y_range=(0, 1024),plot_width=640, plot_height=512,tools=TOOLS)
p1.image(image=[image], x=0, y=0, dh=1024, dw=1280, color_mapper=cmap)
p = figure(x_range=(0, 1280), y_range=(0, 1024),plot_width=640, plot_height=512,tools=TOOLS)
p.image(image=[np.array(ii)], x=0, y=0, dh=1024, dw=1280, color_mapper=cmap)


#p.circle(x=objs['x'],y=objs['y'],radius=6*objs['r'],color="red",fill_color=None)


show(column(p1,p))

NameError: name 'image' is not defined

In [54]:
ii.